---
# **RUN on dataset**
---

In [1]:
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.analysis.stats import StatAnalysis
from pyVHR.plot.visualize import *
import os
import pandas as pd
import plotly.express as px
from numpy import round as r
import plotly.io as pio

# set the right renderer for plotly: 'jupyterlab', 'colab', etc
pio.renderers.default = 'jupyterlab'

# main functions 

def ANALYSIS(dataset, approach, metrics=['MAE'], j_data=False, writeFile=False):
  """Print boxplot analysis"""

  cd = os.getcwd() # current dir
  if not j_data:
    print('## Dataset: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset + "_" + approach + ".h5"
    print('   h5      :', path)
    cfg = path_results + dataset + "_" + approach + ".cfg"
    print('   cfg     :', cfg, end='\n\n')
  else:
    print('## Path dir: ' + dataset)
    print('   Approach: ' + approach)
    path = path_results + dataset 
    print('   path  = ', path)

  filenameH5 = os.path.join(cd, path)
  
  # boxplot
  for m in metrics:
    st = StatAnalysis(filenameH5, join_data=j_data)
    if m=='MAE': 
      scale = 'log' 
    else: 
      scale = 'linear'
    fig = st.displayBoxPlot(metric=m, scale=scale)
    fig.show()
    if writeFile:
      filenamePNG = path_results + dataset + "_" + approach + "_" + m + "_MAE.png"
      fig.write_image(filenamePNG)  

def STATS(dataset, approach='clustering'):
  """Do stats analysis"""
  
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + approach + ".h5"
  df = pd.read_hdf(os.path.join(cd, path))

  # compute errors
  MAE_mean, MAE_std, MAE_median =    r(df['MAE'].mean()[0],2),  r(df['MAE'].std(),2),  r(df['MAE'].median(),2)
  RMSE_mean, RMSE_std, RMSE_median = r(df['RMSE'].mean()[0],2), r(df['RMSE'].std(),2), r(df['RMSE'].median(),2)
  PCC_mean, PCC_std, PCC_median =    r(df['PCC'].mean()[0],2),  r(df['PCC'].std(),2),  r(df['PCC'].median(),2)
  CCC_mean, CCC_std, CCC_median =    r(df['CCC'].mean()[0],2),  r(df['CCC'].std(),2),  r(df['CCC'].median(),2)
  SNR_mean, SNR_std, SNR_median =    r(df['SNR'].mean()[0],2),  r(df['SNR'].std(),2),  r(df['SNR'].median(),2)
  MAX_mean, MAX_std, MAX_median =    r(df['MAX'].mean()[0],2),  r(df['MAX'].std(),2),  r(df['MAX'].median(),2)

 # print table 
  table = [[MAE_mean, MAE_std, MAE_median], [RMSE_mean, RMSE_std, RMSE_median], [PCC_mean, PCC_std, PCC_median],
           [CCC_mean, CCC_std, CCC_median], [SNR_mean, SNR_std, SNR_median], [MAX_mean, MAX_std, MAX_median]]
  stats = pd.DataFrame(table, columns = [ 'MEANS', 'STD', 'MEDIANS'], index=['MAE', 'RMSE', 'PCC', 'CCC', 'SNR', 'MAX'])
  print('\n** Stats table **')
  print(stats)


def RUN(dataset, method):
  """Run methods on a dataset"""

  # set paths
  print('## Dataset: ' + dataset)
  print('Using...')
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('   filename h5: ', path)
  cfg = path_results + dataset + "_" + method + ".cfg"
  print('   cfg        :', cfg)
  print('\n')

  # pipeline
  pl = Pipeline()
  res = pl.run_on_dataset(os.path.join(cd, cfg), verb=1)
  res.saveResults(filenameH5)
  print('Written file: ' + filenameH5 + '\n\n')

def DETAILS(dataset, method, metric='MAE'):
  """Show performance details on each video of the dataset"""

  # set paths
  cd = os.getcwd() # current dir
  path = path_results + dataset + "_" + method + ".h5"
  print('## Dataset: ' + dataset)
  print('   filename h5: ', path)
  df = pd.read_hdf(os.path.join(cd, path))
  datasetname = df['dataset'][0].upper()
  methods = set(list(df['method']))

  # loop on methods
  for m in methods:
    print('   method     :', m)
    vals = df[df['method'] == m][metric]
    fvideos = df[df['method'] == m]['videoFilename']
    x = []
    y = []
    for i in vals.index:
      n = fvideos[i]
      n = n[n.rfind('/', 0, n.rfind('/'))+1:n.rfind('/')]
      x.append(n)
      y.append(vals[i][0])
    fig = px.bar(x=x, y=y, title='dataset: ' + datasetname + ' method: ' + m)
    fig.update_xaxes(type='category')
    fig.show()

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
from scipy.stats import median_abs_deviation

print(median_abs_deviation([1, 2, 3, 4, 5]))


1.0


This notebook shows the complete **pipeline** that takes a **dataset** as input and computes an **estimation** of **BPMs** using a prespecified set of **rPPG** **methods**.

Below are the possible choices for the methods.
* **Methods**: `cpu_CHROM`, `cupy_CHROM`, `torch_CHROM`, `cpu_LGI`, `cpu_POS`, `cupy_POS`, `cpu_PBV`, `cpu_PCA`, `cpu_GREEN`, `cpu_OMIT`, `cpu_ICA`, `cpu_SSR`

Below are the possible choices for the datasets.
* **Datasets**: `PURE`, `UBFC1`, `UBFC2`, `ECG_Fitness_01-1`, `ECG_Fitness_01-2`... `ECG_Fitness_06-1`, `ECG_Fitness_06-2`

# Runs

In [3]:
# run on dataset

path_results = "results/cfg/" # general path for cfg

#dataset = 'PURE'
#RUN(dataset, 'holistic')
#RUN(dataset, 'median')
#RUN(dataset, 'clustering')


# Analisys

In [4]:
## -- Holistic

path_results = "../results/h5/" # general path for cfg# general path for data

dataset = 'UBFC1'
ANALYSIS(dataset, 'holistic', metrics=['MAE'])
STATS(dataset, approach='holistic')

## Dataset: UBFC1
   Approach: holistic
   h5      : ../results/h5/UBFC1_holistic.h5
   cfg     : ../results/h5/UBFC1_holistic.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE    9.97  11.02     3.65
RMSE  16.80  16.56    10.67
PCC    0.47   0.42     0.49
CCC    0.41   0.43     0.27
SNR    3.92   5.24     4.17
MAX   54.67  48.62    45.51


In [5]:
## -- Median
ANALYSIS(dataset, 'median', metrics=['MAE'])
STATS(dataset, approach='median')

## Dataset: UBFC1
   Approach: median
   h5      : ../results/h5/UBFC1_median.h5
   cfg     : ../results/h5/UBFC1_median.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE    1.71   1.91     0.86
RMSE   3.42   3.82     1.76
PCC    0.81   0.23     0.95
CCC    0.76   0.29     0.93
SNR    0.69   2.57     1.28
MAX   15.16  15.21    10.86


In [6]:
## -- Clustering
ANALYSIS(dataset, 'clustering', metrics=['MAE'])
STATS(dataset, approach='clustering')

## Dataset: UBFC1
   Approach: clustering
   h5      : ../results/h5/UBFC1_clustering.h5
   cfg     : ../results/h5/UBFC1_clustering.cfg




** Stats table **
      MEANS   STD  MEDIANS
MAE    0.75  0.35     0.61
RMSE   1.24  1.06     0.80
PCC    0.95  0.08     0.98
CCC    0.94  0.10     0.97
SNR    0.69  2.57     1.28
MAX    5.82  7.73     2.46


In [7]:
## -- Deep 
ANALYSIS(dataset, 'deep', metrics=['MAE'])
STATS(dataset, approach='deep')

## Dataset: UBFC1
   Approach: deep
   h5      : ../results/h5/UBFC1_deep.h5
   cfg     : ../results/h5/UBFC1_deep.cfg




** Stats table **
      MEANS    STD  MEDIANS
MAE    3.33   5.19     1.36
RMSE   5.52   6.85     2.49
PCC    0.66   0.39     0.85
CCC    0.63   0.38     0.81
SNR    5.10   3.49     5.32
MAX   19.71  18.51    13.99


# Details

In [8]:
# holistic

method = 'holistic'
DETAILS(dataset, method, metric='MAE')

## Dataset: UBFC1
   filename h5:  ../results/h5/UBFC1_holistic.h5
   method     : CHROM


   method     : LGI


   method     : ICA


   method     : POS


   method     : PCA


In [9]:
# median
metric='MAE'
method = 'median'
DETAILS(dataset, method, metric)

## Dataset: UBFC1
   filename h5:  ../results/h5/UBFC1_median.h5
   method     : CHROM


   method     : LGI


   method     : ICA


   method     : POS


   method     : PCA


In [10]:
# clustering
metric='MAE'
method = 'clustering'
DETAILS(dataset, method, metric)

## Dataset: UBFC1
   filename h5:  ../results/h5/UBFC1_clustering.h5
   method     : CHROM


   method     : LGI


   method     : ICA


   method     : POS


   method     : PCA


# Multidataset

In [326]:
# multi dataset
#path = path_results + 'h5_collection'
#st = StatAnalysis(path, join_data=True)
#fig = st.displayBoxPlot(metric='PCC')
#fig.show(renderer = "colab")
